# Binary vs MultiClass Classification

Hay alguna gran diferencia o son solo palabros para confundir? La verdad es que hay poca diferencia, normalmente un problema multiclass puede ser tratado como uno binario, y los algoritmos binarios pueden generalizar a multiclase. Así pues, no hay mucho a explicar.

![](https://i.imgur.com/AzAnRJN.jpg)

Quick quiz, quien es quien?

Dicho esto, nos dispondremos a implementar usando sklearn un clasificador de spam. Mejor que el de google? No. Pero bueno, será decent, y será de las primeras veces que salimos a ver datasets reales.

El otro problema que intentaremos solucionar es el clásico y vital problema de saber el origen de una persona dado su nombre. En concreto intentaremos saber si su origen es el planeta Tierra, Una Galaxia Muy Lejana, La Tierra media o El mundo conocido (Westeros y alrededores).



In [21]:
%matplotlib inline

import numpy as np

import pickle
import json
import os
import csv

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from random import sample

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import io

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [0]:
def split_train_val_test(dataset, split=0.2):

    x, y = zip(*dataset)
    x = np.array(list(x))
    y = np.array(list(y))
    sss = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=1337) #l33t seed
    for train_index, test_index in sss.split(x, y):
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
    splits = {'train':(x_train, y_train), 'test':(x_val, y_val)}
    return splits

# Clasification

# Binary Classification Problem

## Planteamiento del problema.

Cuál era el objetivo de hoy? Supervised Learning. Que nos hacía falta? Lo tenemos todo?

![](https://i.imgur.com/3inJ3wG.jpg =500x)

Que preproceso de datos podemos aplicar?

De los tipos de features que hemos visto hasta la fecha, cuales creeis que pueden ser útiles?

De los clasificadores que hemos visto, cuáles pueden ser útiles?

Nos falta algo?

In [1]:
from google.colab import files

In [2]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
df = pd.read_csv(io.StringIO(uploaded['spam.csv'].decode('ISO-8859-1')))
drop_col = df.columns[2:]
df = df.drop(columns=drop_col, axis=1)
df

In [59]:
spam_dataset = []
for index, row in df.iterrows():
    if index>0:
        label = '<NOT_SPAM>' if row[0] == 'ham' else '<SPAM>' 
        sentence = row[1]
        spam_dataset.append((sentence, label))
print(spam_dataset[0])
len(spam_dataset)

('Ok lar... Joking wif u oni...', '<NOT_SPAM>')


5571

### Que datos tenemos?

### Que preproceso?

### Que features?

In [60]:
sample([x for x, y in spam_dataset if y=='<NOT_SPAM>'], 5)

['Horrible bf... I now v hungry...',
 'We have all rounder:)so not required:)',
 'You are everywhere dirt, on the floor, the windows, even on my shirt. And sometimes when i open my mouth, you are all that comes flowing out. I dream of my world without you, then half my chores are out too. A time of joy for me, lots of tv shows i.ll see. But i guess like all things you just must exist, like rain, hail and mist, and when my time here is done, you and i become one.',
 'And you! Will expect you whenever you text! Hope all goes well tomo ',
 'Y lei?']

In [61]:
sample([x for x, y in spam_dataset if y=='<SPAM>'], 5)

["You've won tkts to the EURO2004 CUP FINAL or å£800 CASH, to collect CALL 09058099801 b4190604, POBOX 7876150ppm",
 "Join the UK's horniest Dogging service and u can have sex 2nite!. Just sign up and follow the instructions. Txt ENTRY to 69888 now! Nyt.EC2A.3LP.msg@150p",
 'Adult 18 Content Your video will be with you shortly',
 'Fantasy Football is back on your TV. Go to Sky Gamestar on Sky Active and play å£250k Dream Team. Scoring starts on Saturday, so register now!SKY OPT OUT to 88088',
 'YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call.']

In [0]:
split = split_train_val_test(spam_dataset)

### Pipelines y GridSearch de sklearn.

Las imagenes son super despcriptivas.

![](https://chrisalbon.com/images/machine_learning_flashcards/Hyperparameter_Tuning_print.png =500x)

![](https://cambridgecoding.files.wordpress.com/2016/03/gridsearch_cv.png)

Cómo lo hacemos con sklearn? Ahora lo veremos.

In [63]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()), #MultinomialNB
])
pipeline.get_params().keys() # que parametros podemos tocar en el gridsearch!

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__alpha', 'clf__class_prior', 'clf__fit_prior'])

In [0]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 2), (1, 3), (2, 3), (1,4)),  #ngrams to test
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    #'vect__analyzer': ('word', 'char', 'char_wb')
}

In [0]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [66]:
grid_search.fit(split['train'][0], split['train'][1])
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search.best_score_)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:  1.3min finished


	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__ngram_range: (1, 2)
Best score: 0.935


In [67]:
pipeline.set_params(**best_parameters)
predictions = pipeline.score(split['test'][0], split['test'][1])
print('TEST SCORE: {}'.format(predictions))

TEST SCORE: 0.9515695067264573


In [0]:
is_spam = sample([x for x, y in spam_dataset if y=='<SPAM>'], 5)

In [70]:
p_spam = sample(list(split['test'][0]), 10)
p_spam

['Yes see ya not on the dot',
 "That's necessarily respectful",
 'Gosh that , what a pain. Spose I better come then.',
 'Take something for pain. If it moves however to any side in the next 6hrs see a doctor.',
 "Nobody names their penis a girls name this story doesn't add up at all",
 'at bruce b downs &amp; fletcher now',
 'Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed å£1000 cash or å£5000 prize!',
 'Not tonight mate. Catching up on some sleep. This is my new number by the way. ',
 'Happy birthday to you....dear.with lots of love.rakhesh NRI',
 'ITS A LAPTOP TAKE IT WITH YOU.']

In [73]:
predictions = pipeline.predict(is_spam)
predictions_probs = pipeline.predict_proba(is_spam)
for x, y, p in zip(p_spam, predictions, predictions_probs):
    print('{} ------> {} {}'.format(x, y, p)) # p

Yes see ya not on the dot ------> <SPAM> [0.02281379 0.97718621]
That's necessarily respectful ------> <SPAM> [0.24044046 0.75955954]
Gosh that , what a pain. Spose I better come then. ------> <SPAM> [0.32486223 0.67513777]
Take something for pain. If it moves however to any side in the next 6hrs see a doctor. ------> <SPAM> [0.08823808 0.91176192]
Nobody names their penis a girls name this story doesn't add up at all ------> <SPAM> [0.13111601 0.86888399]


#  Multiclass

![](https://vignette.wikia.nocookie.net/starwars/images/4/42/StarWarsOpeningLogo.svg/revision/latest?cb=20120211213511 =100x) 
![](https://trifectanetworks.files.wordpress.com/2015/10/vs1.gif =50x)
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSWOGmaOAYGuI0Hqs-N0HK82cBRNNVYHKr42Rc_fzNX6guMNeB5 =70x)
![](https://trifectanetworks.files.wordpress.com/2015/10/vs1.gif =50x)
![](https://pmctvline2.files.wordpress.com/2016/04/thrones-title-sequence.jpg?w=620&h=420&crop=1 =100x)
![](https://trifectanetworks.files.wordpress.com/2015/10/vs1.gif =50x)
![](https://www.caracteristicas.co/wp-content/uploads/2017/02/planeta-tierra-min-e1487870067631.jpg =120x)

### Planteamiento del problema

## Planteamiento del problema.

Cuál era el objetivo de hoy? Supervised Learning. Que nos hacía falta? Lo tenemos todo?

![](https://i.imgur.com/3inJ3wG.jpg =500x)

Que preproceso de datos podemos aplicar?

De los tipos de features que hemos visto hasta la fecha, cuales creeis que pueden ser útiles?

De los clasificadores que hemos visto, cuáles pueden ser útiles?

Nos falta algo?


In [7]:
import json

In [3]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving sw_names.json to sw_names.json
Saving nombres.txt to nombres.txt
Saving lotr_names.txt to lotr_names.txt
Saving got_names.json to got_names.json
User uploaded file "sw_names.json" with length 1566 bytes
User uploaded file "nombres.txt" with length 3555 bytes
User uploaded file "lotr_names.txt" with length 14803 bytes
User uploaded file "got_names.json" with length 1586 bytes


In [16]:
sw_names = uploaded['sw_names.json']
sw_names = json.loads(sw_names)
len(sw_names),sw_names[0:10]

(93,
 ['4-LOM',
  'Aayla Secura',
  'Admiral Ackbar',
  'Admiral Thrawn',
  'Ahsoka Tano',
  'Anakin Solo',
  'Asajj Ventress',
  'Aurra Sing',
  'Senator Bail Organa',
  'Barriss Offee'])

In [15]:
got_names = uploaded['got_names.json']
got_names = json.loads(got_names)
len(got_names), got_names[0:10]

(81,
 ['Tyrion Lannister',
  'Jaime Lannister',
  'Cersei Lannister',
  'Daenerys Targaryen',
  'Jon Snow',
  'Petyr Baelish',
  'Jorah Mormont',
  'Sansa Stark',
  'Arya Stark',
  'Theon Greyjoy'])

In [31]:
common_names = uploaded['nombres.txt']
common_names = io.StringIO(common_names.decode('utf-8')).getvalue()
common_names = common_names.split('\n')
len(common_names), common_names[0:10]

(456,
 ['Aarón',
  'Abdón',
  'Abel',
  'Abelardo',
  'Abrahán',
  'Absalón',
  'Acacio',
  'Adalberto',
  'Adán',
  'Adela'])

In [37]:
lotr_names = uploaded['lotr_names.txt']
lotr_names = io.StringIO(lotr_names.decode('utf-8')).getvalue()
lotr_names = [pair.split(',')[0].rstrip() for pair in lotr_names.split('\n')]
len(lotr_names), lotr_names[0:10]

(828,
 [' Aragorn II',
  ' Arwen',
  ' Elrond',
  ' Celebrían',
  ' Elrohir',
  ' Elladan',
  ' Turgon',
  ' Elenwë',
  ' Manwë',
  ' Ulmo'])

In [39]:
labels = ['lotr', 'common', 'got', 'sw']
data = []
for label, names in zip(labels, [lotr_names, common_names, got_names, sw_names]):
    data += [(n, label) for n in names]
len(data)

1458

In [0]:
split = split_train_val_test(data)

In [0]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char')),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(class_weight='balanced', probability=True)), #MultinomialNB
])
pipeline.get_params().keys() # que parametros podemos tocar en el gridsearch!

dict_keys(['vect__encoding', 'clf__cache_size', 'vect__input', 'clf__gamma', 'vect__dtype', 'vect__max_features', 'vect__token_pattern', 'clf__kernel', 'tfidf', 'vect__vocabulary', 'clf__random_state', 'clf__max_iter', 'vect__max_df', 'vect__ngram_range', 'clf__coef0', 'vect__analyzer', 'tfidf__norm', 'vect', 'vect__stop_words', 'clf__class_weight', 'vect__min_df', 'vect__tokenizer', 'clf__decision_function_shape', 'vect__preprocessor', 'clf__verbose', 'clf__degree', 'tfidf__use_idf', 'clf__probability', 'clf__shrinking', 'vect__binary', 'steps', 'vect__decode_error', 'tfidf__smooth_idf', 'clf__C', 'clf__tol', 'clf', 'vect__strip_accents', 'tfidf__sublinear_tf', 'vect__lowercase'])

In [0]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 2), (1, 3), (2, 3), (1,4)),  #ngrams to test
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__kernel': ('linear', 'rbf'),
    'clf__degree': (3, 4, 5),
    'clf__C': (0.1, 0.5, 1.0)
}

In [0]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [0]:
grid_search.fit(split['train'][0], split['train'][1])
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search.best_score_)

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 16.9min
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed: 18.1min finished


	clf__C: 1.0
	clf__degree: 3
	clf__kernel: 'linear'
	tfidf__norm: 'l2'
	tfidf__use_idf: False
	vect__max_df: 0.75
	vect__ngram_range: (1, 4)
Best score: 0.937


In [0]:
pipeline.set_params(**best_parameters)

Pipeline(steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.75, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None, stop_words=None,
        stri...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [0]:
#pipeline.steps[0][1].vocabulary_

In [0]:
predictions = pipeline.score(split['test'][0], split['test'][1])
print('TEST SCORE: {}'.format(predictions))

TEST SCORE: 0.952054794520548


In [0]:
p_names = ['David', 'Luisa', 'Eragorn I', 'Obe One', 'Wen Kenoeb', 'Madre Teresa', 'John Storm', 'Darya Stormborn']

In [0]:
predictions = pipeline.predict(p_names)
predictions_probs = pipeline.predict_proba(p_names)
for x, y, p in zip(p_names, predictions, predictions_probs):
    print('{} ------> {}'.format(x, y, p)) # p

David ------> common
Luisa ------> common
Eragorn I ------> lotr
Obe One ------> sw
Wen Kenoeb ------> sw
Madre Teresa ------> got
John Storm ------> got
Darya Stormborn ------> got
